In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [63]:
import ipy_progressbar

In [5]:
import re

In [36]:
STMSTOPS = "data/stops.txt"
BASICBIZ = "data/basic_yelp_business_data.csv"

In [3]:
stops = pd.read_csv(STMSTOPS)

In [18]:
metro_re = re.compile(r'http://www\.stm\.info/metro/')
bus_re = re.compile(r'http://m\.stm\.info/fr/horaires/arrets/\d+/bus')

In [19]:
imetro = stops.stop_url.str.match(metro_re)

In [34]:
stop_type = []
for url in stops.stop_url:
    if metro_re.match(url):
        stop_type.append('metro')
    elif bus_re.match(url):
        stop_type.append('bus')
    else:
        stop_type.append(None)
stops['stop_type'] = stop_type
stops['stop_type'] = stops['stop_type'].astype('category')

In [56]:
stops.head()

,stop_id,stop_code,stop_name,stop_lat,stop_lon,stop_url,wheelchair_boarding,stop_type
0,1,10280,Station Henri-Bourassa,45.556109,-73.667452,http://www.stm.info/metro/M01.htm,1,metro
1,10,10262,Station Sherbrooke,45.518345,-73.568251,http://www.stm.info/metro/M10.htm,2,metro
2,100006,61396,Sainte-Anne / Sainte-Marie,45.402742,-73.947489,http://m.stm.info/fr/horaires/arrets/61396/bus,2,bus
3,100007,61503,Robert / Cluny,45.580406,-73.607774,http://m.stm.info/fr/horaires/arrets/61503/bus,1,bus
4,100009,61506,du Bois-de-Boulogne / stationnement,45.541494,-73.689973,http://m.stm.info/fr/horaires/arrets/61506/bus,2,bus


In [39]:
business = pd.read_csv(BASICBIZ, index_col='business_id')

In [42]:
business.head()

,state,longitude,latitude,name,stars
business_id,,,,,
lW9Up8oVTWUEut_R6nZG-g,QC,-73.544570,45.545073,Atomic Café,4.0
rWmYk-i8EF-g07KnVkdlqQ,QC,-73.542120,45.561749,Restaurant Les Princesses,2.5
RZma06CqXbOIMzTQL-eJFQ,QC,-73.635581,45.589137,La Cage Aux Sports,2.0
GxfgoiDLfs7HC8C_wFsGPw,QC,-73.574577,45.548671,Boulangerie Première Moisson,4.5
e1_MTuBa4LTN1E2RKh53Xg,QC,-73.585389,45.555818,Zytynsky's Deli,4.5


In [74]:
ismetro = stops['stop_type'] == 'metro'
metros = stops[ismetro]
isbus = stops['stop_type'] == 'bus'
buses = stops[isbus]

In [75]:
mindist = []
for lon, lat in ipy_progressbar.ProgressBar(zip(business.longitude, business.latitude)):
    result = []
    for stp in [metros, buses]:
        dlon = lon - stp.stop_lon
        dlat = lat - stp.stop_lat
        dists = np.sqrt(dlon**2 + dlat**2)
        imin = np.argmin(dists)
        result.extend((dists[imin], stp.stop_id.ix[imin], stp.stop_type.ix[imin]))
    mindist.append(result)

In [76]:
stm_prox = pd.DataFrame(mindist, columns=('closest_mstop_dist', 'closest_mstop_id', 'closest_mstop_type', 'closest_bstop_dist', 'closest_bstop_id', 'closest_bstop_type'), index=business.index)

In [78]:
stm_prox.head()

,closest_mstop_dist,closest_mstop_id,closest_mstop_type,closest_bstop_dist,closest_bstop_id,closest_bstop_type
business_id,,,,,,
lW9Up8oVTWUEut_R6nZG-g,0.007192,25,metro,0.000308,120961,bus
rWmYk-i8EF-g07KnVkdlqQ,0.005166,23,metro,0.000289,121211,bus
RZma06CqXbOIMzTQL-eJFQ,0.043264,3,metro,0.000325,1800551,bus
GxfgoiDLfs7HC8C_wFsGPw,0.021392,26,metro,0.000157,115294,bus
e1_MTuBa4LTN1E2RKh53Xg,0.015219,64,metro,0.000505,113214,bus


In [79]:
stm_prox.to_csv("data/closest_bus-metro_stops.csv")

In [ ]:
!less data/closest_bus-metro_stops.csv

business_id,closest_mstop_dist,closest_mstop_id,closest_mstop_type,closest_bstop _dist,closest_bstop_id,closest_bstop_type
lW9Up8oVTWUEut_R6nZG-g,0.00719181923163,25,metro,0.000308031752548,120961,bus
rWmYk-i8EF-g07KnVkdlqQ,0.00516627381873,23,metro,0.000289003754303,121211,bus
RZma06CqXbOIMzTQL-eJFQ,0.0432638167741,3,metro,0.000325377104303,1800551,bus
GxfgoiDLfs7HC8C_wFsGPw,0.0213923534101,26,metro,0.000156605300042,115294,bus
e1_MTuBa4LTN1E2RKh53Xg,0.015218758449,64,metro,0.000505095367232,113214,bus
yLj-pKIc-V530j1G6ySQ2Q,0.00601657463346,22,metro,0.000636730963596,119900,bus
IY5GFZTZCywC0ylKuG9ebA,0.00504357888112,26,metro,0.000849245977327,118724,bus
g8Ja8BKskzmWHX9VH7B2yg,0.0144520787667,24,metro,0.00020388741992,101260,bus
Env21pieyFD60kaC0xbmcg,0.00142186768723,20,metro,8.7285279386e-05,121204,bus
RySnetMQUJKbbtliTt4Nmw,0.002806483495,23,metro,0.00255223643837,142122,bus
IYdjcA92wjvM5RtPZ3C7kw,0.0230580935248,19,metro,0.00260055859384,1602252,bus
pvbBfLIVJ_5sJXUTAKmkng,0.0284